In [10]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, List, Set


@dataclass(frozen=True)
class Requirement:
    id: str
    description: str


SAMPLE_REQUIREMENTS: Dict[str, Requirement] = {
    "R1": Requirement("R1", "User login"),
    "R2": Requirement("R2", "Password reset"),
    "R3": Requirement("R3", "Multi-factor authentication"),
    "R4": Requirement("R4", "Security log monitoring"),
    "R5": Requirement("R5", "Admin dashboard"),
}

SAMPLE_DEPENDENCIES: List[tuple[str, str]] = [
    ("R1", "R2"),  
    ("R1", "R3"),
    ("R3", "R4"),
    ("R1", "R5"),
    ("R3", "R5"),
]


def build_graph(
    deps: List[tuple[str, str]], *, reverse: bool = False
) -> Dict[str, Set[str]]:
    graph: Dict[str, Set[str]] = {}
    for src, dst in deps:
        a, b = (dst, src) if reverse else (src, dst)
        graph.setdefault(a, set()).add(b)
        graph.setdefault(b, set()) 
    return graph


def dfs_reachable(graph: Dict[str, Set[str]], start: str) -> Set[str]:
    visited: Set[str] = set()
    stack: List[str] = [start]
    while stack:
        node = stack.pop()
        for nxt in graph.get(node, ()):
            if nxt not in visited:
                visited.add(nxt)
                stack.append(nxt)
    visited.discard(start)
    return visited


def format_impact(impacted: Set[str]) -> str:
    if not impacted:
        return "No other requirements are impacted."
    lines = ["Impacted requirements:"]
    for rid in sorted(impacted):
        req = SAMPLE_REQUIREMENTS.get(rid)
        if req:
            lines.append(f"- {req.id}: {req.description}")
        else:
            lines.append(f"- {rid}: <unknown requirement>")
    return "\n".join(lines)


def print_tree(graph: Dict[str, Set[str]], node: str, allowed: Set[str], indent: str = "", seen: Set[str] | None = None, is_last: bool = True) -> None:
    if seen is None:
        seen = set()
    seen.add(node)

    label = SAMPLE_REQUIREMENTS.get(node)
    text = f"{node}"
    if label:
        text += f" ({label.description})"

    connector = "└─ " if indent else ""
    print(f"{indent}{connector}{text}")

    children = [
        c for c in sorted(graph.get(node, ()))
        if c in allowed and c not in seen
    ]

    for index, child in enumerate(children):
        last = index == len(children) - 1
        if indent:
            child_indent = indent + ("   " if is_last else "│  ")
        else:
            child_indent = "   "  
        print_tree(graph, child, allowed, child_indent, seen, last)

def run_cli() -> None:
    print("Available requirements:")
    for req in SAMPLE_REQUIREMENTS.values():
        print(f"- {req.id}: {req.description}")

    changed = input("\nEnter changed requirement ID (e.g., R1): ").strip().upper()
    if changed not in SAMPLE_REQUIREMENTS:
        print(f"Unknown requirement: {changed}")
        return

    mode = input("Impact direction [F=forward (default) / B=backward]: ").strip().upper()
    reverse = mode == "B"

    graph = build_graph(SAMPLE_DEPENDENCIES, reverse=reverse)
    impacted = dfs_reachable(graph, changed)

    print("\nChange type:", "Downstream impact" if not reverse else "Upstream impact")
    print(format_impact(impacted))

    show_tree = input("\nShow ASCII dependency tree? [Y/n]: ").strip().lower()
    if show_tree in ("", "y", "yes"):
        print("\nDependency view:")
        allowed = set(impacted)
        allowed.add(changed)
        print_tree(graph, changed, allowed)

if __name__ == "__main__":
    run_cli()


Available requirements:
- R1: User login
- R2: Password reset
- R3: Multi-factor authentication
- R4: Security log monitoring
- R5: Admin dashboard



Enter changed requirement ID (e.g., R1):  R1
Impact direction [F=forward (default) / B=backward]:  F



Change type: Downstream impact
Impacted requirements:
- R2: Password reset
- R3: Multi-factor authentication
- R4: Security log monitoring
- R5: Admin dashboard



Show ASCII dependency tree? [Y/n]:  Y



Dependency view:
R1 (User login)
   └─ R2 (Password reset)
   └─ R3 (Multi-factor authentication)
   │  └─ R4 (Security log monitoring)
   │  └─ R5 (Admin dashboard)
   └─ R5 (Admin dashboard)
